# Log Scale

In [1]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
from IPython.display import display

In [3]:
spark_application_name = "Spark_Application_Name"

In [4]:
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/13 10:26:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/13 10:26:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/13 10:26:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
filePath = "sf-stocks-clean.parquet"
stocksDF = spark.read.parquet(filePath)
(trainDF, testDF) = stocksDF.randomSplit([.8, .2], seed=42)

In [6]:
from pyspark.sql.functions import col, log
from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.regression import LinearRegression

logTrainDF = trainDF.withColumn("log_low", log(col("Low")))
logTestDF = testDF.withColumn("log_low", log(col("Low")))

rFormula = RFormula(formula="log_low ~ . - Low", featuresCol="features", labelCol="log_low", handleInvalid="skip")

lr = LinearRegression(labelCol="log_low", predictionCol="log_pred")
pipeline = Pipeline(stages = [rFormula, lr])
pipelineModel = pipeline.fit(logTrainDF)
predDF = pipelineModel.transform(logTestDF)

22/06/13 10:29:21 WARN Instrumentation: [8c62c44a] regParam is zero, which might cause numerical instability and overfitting.


## Exponentiate

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, exp

expDF = predDF.withColumn("prediction", exp(col("log_pred")))

regressionEvaluator = RegressionEvaluator(labelCol="Low", predictionCol="prediction")
rmse = regressionEvaluator.setMetricName("rmse").evaluate(expDF)
r2 = regressionEvaluator.setMetricName("r2").evaluate(expDF)
print(f"RMSE is {rmse}")
print(f"R2 is {r2}")

RMSE is 263.4108902331179
R2 is 0.8672557055279959
